In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
import random
import requests

def sample_books(book_urls,labels):
    # Create a list to store the DataFrames
    dataframes = []
    # Iterate over the book URLs and labels
    for url,label in zip(book_urls,labels):
        # Download book from Gutenberg
        book = requests.get(url)
        book = book.text
        # Split book into a list of words
        words = book.split()
        # Create a list to store the partitions
        partitions = []
        # Use a for loop to sample 200 partitions of 150 words each
        for i in range(200):
            start_index = random.randint(0, len(words) - 150)
            partition = words[start_index : start_index + 150]
            partitions.append(" ".join(partition))
        # Create a Pandas DataFrame to store the partitions
        data = {'text': partitions,'book':label}
        df = pd.DataFrame(data)
        # Append the DataFrame to the list
        dataframes.append(df)
    # Concatenate the DataFrames into a single DataFrame
    final_df = pd.concat(dataframes)
    #return dataframe
    return final_df

book_urls = ['https://www.gutenberg.org/files/145/145-0.txt', 'https://www.gutenberg.org/files/14255/14255-8.txt','https://www.gutenberg.org/files/16807/16807-8.txt','https://www.gutenberg.org/files/20289/20289-8.txt','https://www.gutenberg.org/files/17699/17699-8.txt']

labels = ['a' , 'b', 'c', 'd', 'e']
df = sample_books(book_urls,labels)
df.to_csv('text_data2.csv',index=False)
# Load the data into a pandas dataframe
df = pd.read_csv("text_data2.csv")

In [3]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.manifold import TSNE
import gensim.downloader as api

# Load the data into a pandas dataframe
df = pd.read_csv("text_data2.csv")

# Preprocess the data
df['text'] = df['text'].str.lower() # Convert to lowercase
df['text'] = df['text'].str.replace('[^a-zA-Z0-9\s]','') # Remove punctuation 

C:\Users\joshr\AppData\Local\Temp\ipykernel_9096\622637413.py:14: FutureWarning: The default value of regex will change from True to False in a future version.
  df['text'] = df['text'].str.replace('[^a-zA-Z0-9\s]','') # Remove punctuation


In [4]:
# Bag of Words
vectorizer = CountVectorizer(max_features=5000)
bow = vectorizer.fit_transform(df['text'])
bow_df = pd.DataFrame(bow.toarray(), columns=vectorizer.get_feature_names())

e:\Softwares\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [5]:
# TF-IDF
vectorizer = TfidfVectorizer(max_features=5000)
tfidf = vectorizer.fit_transform(df['text'])
tfidf_df = pd.DataFrame(tfidf.toarray(), columns=vectorizer.get_feature_names())

e:\Softwares\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [6]:
# LDA
vectorizer = CountVectorizer(max_features=5000)
lda_input = vectorizer.fit_transform(df['text'])
lda = LatentDirichletAllocation(n_components=10)
lda_output = lda.fit_transform(lda_input)
lda_df = pd.DataFrame(lda_output, columns=['topic'+str(i) for i in range(lda.n_components)])

In [7]:
# Word Embeddings
model = api.load('glove-wiki-gigaword-100')
embeddings = []
for doc in df['text']:
    doc_embeddings = []
    for word in doc.split():
        if word in model:
            doc_embeddings.append(model[word])
    if len(doc_embeddings) > 0:
        embeddings.append(np.mean(doc_embeddings, axis=0))
    else:
        embeddings.append(np.zeros(100))
embedding_df = pd.DataFrame(embeddings, columns=['embedding_'+str(i) for i in range(100)])

# Save the results to csv files
bow_df.to_csv('bow.csv', index=False)
tfidf_df.to_csv('tfidf.csv', index=False)
lda_df.to_csv('lda.csv', index=False)
embedding_df.to_csv('embeddings.csv', index=False)